# Chat Client 사용

이 노트북은 Azure AI Agent Framework를 사용하여 채팅 클라이언트를 구축하는 방법을 단계별로 안내합니다.

## 학습 목표
- Azure AI Agent Framework의 핵심 개념 이해
- 다양한 Azure 클라이언트 (AzureAIAgentClient, AzureOpenAIChatClient, AzureOpenAIAssistantsClient) 사용법
- Function Calling을 통한 외부 함수 연동
- 스트리밍 응답 처리
- 응답 취소 메커니즘

## 사전 요구사항
1. **uv 설치** (`curl -LsSf https://astral.sh/uv/install.sh | sh`)
2. **Python 3.12 이상**
3. **Azure CLI 설치 및 로그인** (`az login`)
4. **Azure OpenAI 또는 Azure AI 서비스 리소스**
5. **uv를 통한 Python 패키지 관리** (pyproject.toml 기반)

## 환경 설정
이 프로젝트는 uv를 사용하여 Python 환경과 의존성을 관리합니다:
- `pyproject.toml`: 프로젝트 의존성 정의 (Python 3.12+ 요구)
- `uv.lock`: 정확한 버전 고정
- `uv sync --dev`: 개발 환경 의존성 설치

## 1. 환경 설정 및 패키지 설치

uv를 사용하여 필요한 패키지들을 설치합니다. 

**첫 실행 시 터미널에서 다음 명령어를 실행하세요:**
```bash
# Python 3.12로 가상환경 생성
uv venv --python 3.12

# 프로젝트 의존성 설치
uv sync --dev

# Jupyter 커널에 가상환경 등록 (선택사항)
uv run python -m ipykernel install --user --name maf-workshop
```

In [1]:
# uv를 사용한 패키지 설치
import subprocess
import sys
import os

def install_with_uv():
    """uv를 사용하여 패키지를 설치합니다."""
    try:
        # 현재 작업 디렉토리 확인
        current_dir = os.getcwd()
        print(f"현재 작업 디렉토리: {current_dir}")
        
        # pyproject.toml 파일이 있는지 확인
        pyproject_path = os.path.join(current_dir, "pyproject.toml")
        if os.path.exists(pyproject_path):
            print("✅ pyproject.toml 파일을 찾았습니다.")
            
            # uv sync로 의존성 설치
            print("📦 uv sync를 실행하여 의존성을 설치합니다...")
            result = subprocess.run(
                ["uv", "sync", "--dev"],
                capture_output=True,
                text=True,
                cwd=current_dir
            )
            
            if result.returncode == 0:
                print("✅ uv sync 완료!")
                print("설치된 패키지 목록 (일부):")
                
                # 주요 패키지 확인
                key_packages = [
                    "agent-framework",
                    "agent-framework-azure-ai", 
                    "python-dotenv",
                    "azure-identity",
                    "pydantic"
                ]
                
                list_result = subprocess.run(
                    ["uv", "pip", "list"],
                    capture_output=True,
                    text=True,
                    cwd=current_dir
                )
                
                if list_result.returncode == 0:
                    installed_packages = list_result.stdout
                    for pkg in key_packages:
                        if pkg in installed_packages:
                            print(f"  ✅ {pkg}")
                        else:
                            print(f"  ❌ {pkg}")
                
            else:
                print(f"❌ uv sync 실패: {result.stderr}")
                # pip으로 대체 설치 시도
                print("🔄 pip으로 대체 설치를 시도합니다...")
                fallback_install()
                
        else:
            print("❌ pyproject.toml 파일을 찾을 수 없습니다.")
            print("🔄 pip으로 개별 패키지를 설치합니다...")
            fallback_install()
            
    except FileNotFoundError:
        print("❌ uv가 설치되지 않았습니다.")
        print("💡 uv 설치 방법: curl -LsSf https://astral.sh/uv/install.sh | sh")
        print("🔄 pip으로 개별 패키지를 설치합니다...")
        fallback_install()
    except Exception as e:
        print(f"❌ 예기치 않은 오류: {e}")
        fallback_install()

def fallback_install():
    """pip을 사용한 대체 설치"""
    packages = [
        "agent-framework",
        "agent-framework-azure-ai",
        "python-dotenv",
        "azure-identity",
        "pydantic"
    ]
    
    for package in packages:
        try:
            print(f"📦 {package} 설치 중...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✅ {package} 설치 완료")
        except subprocess.CalledProcessError:
            print(f"❌ {package} 설치 실패")

# 패키지 설치 실행
install_with_uv()

현재 작업 디렉토리: /Users/andy/works/ai/maf-workshop
✅ pyproject.toml 파일을 찾았습니다.
📦 uv sync를 실행하여 의존성을 설치합니다...
✅ uv sync 완료!
설치된 패키지 목록 (일부):
  ✅ agent-framework
  ✅ agent-framework-azure-ai
  ✅ python-dotenv
  ✅ azure-identity
  ✅ pydantic


## 2. 필요한 라이브러리 임포트

Azure AI Agent Framework와 관련 라이브러리들을 임포트합니다.

In [2]:
# Copyright (c) Microsoft. All rights reserved.

import asyncio
import os
from random import randint
from typing import Annotated

# Agent Framework 라이브러리
from agent_framework import ChatResponse
from agent_framework.azure import (
    AzureAIAgentClient,
    AzureOpenAIChatClient,
    AzureOpenAIAssistantsClient,
    AzureOpenAIResponsesClient
)

# Azure 인증 (AzureAIAgentClient용)
from azure.identity.aio import AzureCliCredential as AsyncAzureCliCredential

# 유틸리티
from pydantic import BaseModel, Field

print("✅ 모든 라이브러리 임포트 완료!")


✅ 모든 라이브러리 임포트 완료!


## 3. 환경 변수 로드

`.env` 파일에서 Azure 설정 정보를 로드합니다.

`.env` 파일 예시:
```
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/
AZURE_OPENAI_API_KEY=your-api-key
AZURE_OPENAI_API_VERSION=2024-02-01
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME=gpt-4
```

**참고:** 이 노트북은 API 키 방식의 인증을 사용합니다. `AZURE_OPENAI_API_KEY` 환경 변수가 필수입니다.

In [3]:
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv(override=True)

# 환경 변수 확인
print("환경 변수 확인:")
env_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY",
    "AZURE_OPENAI_API_VERSION",
    "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"
]

for var in env_vars:
    value = os.getenv(var)
    if value:
        # API 키는 부분적으로만 표시
        if "API_KEY" in var:
            print(f"✅ {var}: {value[:8]}...")
        else:
            print(f"✅ {var}: {value}")
    else:
        print(f"❌ {var}: 설정되지 않음")

# API 키 확인
api_key = os.getenv("AZURE_OPENAI_API_KEY")
if not api_key:
    print("\n⚠️ 경고: AZURE_OPENAI_API_KEY 환경 변수가 설정되지 않았습니다.")
    print("클라이언트 생성 시 오류가 발생할 수 있습니다.")
else:
    print("\n✅ API 키가 설정되었습니다. 클라이언트를 사용할 준비가 완료되었습니다.")

환경 변수 확인:
✅ AZURE_OPENAI_ENDPOINT: https://aistudioaiservices343274518006.openai.azure.com
✅ AZURE_OPENAI_API_KEY: 6hBhIb88...
✅ AZURE_OPENAI_API_VERSION: 2024-08-01-preview
✅ AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: gpt-4.1

✅ API 키가 설정되었습니다. 클라이언트를 사용할 준비가 완료되었습니다.


## 4. 함수 도구 정의

AI 모델이 호출할 수 있는 사용자 정의 함수를 정의합니다. Function Calling을 통해 AI가 외부 데이터나 서비스에 접근할 수 있습니다.

In [4]:
def get_weather(
    location: Annotated[str, Field(description="날씨를 조회할 도시 이름")],
) -> str:
    """지정된 위치의 날씨 정보를 반환합니다."""
    conditions = ["맑음", "흐림", "비", "눈", "폭풍"]
    temperature = randint(10, 30)
    condition = conditions[randint(0, len(conditions)-1)]
    return f"{location}의 날씨는 {condition}이며, 최고 기온은 {temperature}°C입니다."

def get_time_in_city(
    city: Annotated[str, Field(description="시간을 조회할 도시 이름")],
) -> str:
    """특정 도시의 현재 시간을 반환합니다 (시뮬레이션)."""
    from datetime import datetime, timedelta
    import random
    
    # 임의의 시차를 적용한 시간 생성 (실제 API 대신)
    offset_hours = random.randint(-12, 12)
    city_time = datetime.now() + timedelta(hours=offset_hours)
    return f"{city}의 현재 시간은 {city_time.strftime('%Y-%m-%d %H:%M:%S')}입니다."

# 구조화된 출력을 위한 모델 정의
class WeatherInfo(BaseModel):
    """날씨 정보의 구조화된 출력 모델"""
    location: str = Field(description="도시 이름")
    condition: str = Field(description="날씨 상태")
    temperature: int = Field(description="온도 (섭씨)")
    description: str = Field(description="날씨 설명")

print("✅ 함수 도구 정의 완료!")
print("📝 정의된 함수:")
print("  - get_weather(): 날씨 조회")
print("  - get_time_in_city(): 도시별 시간 조회")
print("  - WeatherInfo: 구조화된 날씨 정보 모델")

✅ 함수 도구 정의 완료!
📝 정의된 함수:
  - get_weather(): 날씨 조회
  - get_time_in_city(): 도시별 시간 조회
  - WeatherInfo: 구조화된 날씨 정보 모델


## 5. Azure AI Agent Client 사용하기

`AzureAIAgentClient`는 Azure AI 서비스와 상호작용하는 가장 직접적인 방법입니다.

In [5]:
async def demo_azure_ai_agent_client():
    """Azure AI Agent Client 기본 사용법을 시연합니다."""
    
    print("🚀 Azure AI Agent Client 데모 시작...")
    
    try:
        # Azure CLI 자격 증명을 사용하여 클라이언트 생성
        async with AzureAIAgentClient(async_credential=AsyncAzureCliCredential()) as client:
            
            # 1. 간단한 채팅
            print("\n--- 1. 기본 채팅 ---")
            message = "안녕하세요! Azure AI Agent Framework에 대해 간단히 설명해주세요."
            print(f"사용자: {message}")
            
            response = await client.get_response(message)
            print(f"AI: {response}")
            
            # 2. Function Calling 사용
            print("\n--- 2. Function Calling ---")
            message = "암스테르담과 파리의 날씨를 알려주세요."
            print(f"사용자: {message}")
            
            response = await client.get_response(message, tools=get_weather)
            print(f"AI: {response}")
            
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print("💡 AZURE_OPENAI_API_KEY 환경 변수를 확인해주세요.")

# 비동기 함수 실행
await demo_azure_ai_agent_client()

🚀 Azure AI Agent Client 데모 시작...

--- 1. 기본 채팅 ---
사용자: 안녕하세요! Azure AI Agent Framework에 대해 간단히 설명해주세요.
AI: 안녕하세요!  
**Azure AI Agent Framework**는 Microsoft Azure에서 AI 기반 에이전트(Agent)를 쉽고 효율적으로 개발, 배포, 관리할 수 있도록 지원하는 프레임워크입니다.

### 주요 특징
1. **에이전트 기반 아키텍처**  
   자연어 처리(NLP), 의사결정, 툴 연동 등 다양한 기능을 가진 에이전트(보통 LLM 활용)를 개발할 수 있는 구조를 제공합니다.

2. **워크플로우 관리**  
   다양한 작업(Task)을 효과적으로 분리하고, 워크플로우(Workflow)를 통해 복잡한 시나리오에 대응할 수 있습니다.

3. **플러그인 및 툴 통합**  
   검색, 데이터베이스 질의, 외부 API 호출 등 다양한 도구(Tool)와 쉽게 통합할 수 있습니다.

4. **Azure 서비스와 연계**  
   Azure OpenAI, Cognitive Search, Functions 등 Azure의 다양한 AI 및 클라우드 서비스와 연동이 쉽도록 설계되어 있습니다.

5. **엔터프라이즈 지원**  
   보안, 모니터링, 배포 자동화 등 기업 환경에서 필요한 기능을 지원합니다.

### 대표적인 활용 예시
- 지식 기반 챗봇 구축  
- 업무 자동화 에이전트  
- 고객 지원/상담 Agent  
- 내부 정보 검색 Agent 등

---

#### 참고 링크
- [공식 Microsoft 문서: Azure AI Agent Framework](https://learn.microsoft.com/en-us/azure/ai-services/agent-framework/overview)

필요하시다면 더 상세한 아키텍처나 사용 예시도 설명드릴 수 있습니다!

--- 2. Function Calling ---
사용자: 암스테르담과 파리의 

## 6. Azure OpenAI Chat Client 사용하기

`AzureOpenAIChatClient`는 Azure OpenAI 서비스를 직접 사용하는 클라이언트입니다.

In [6]:
async def demo_azure_openai_chat_client():
    """Azure OpenAI Chat Client 사용법을 시연합니다."""
    
    print("🚀 Azure OpenAI Chat Client 데모 시작...")
    
    try:
        # 환경 변수에서 API 키를 읽어 클라이언트 생성
        client = AzureOpenAIChatClient()
        
        # 1. 기본 채팅
        print("\n--- 1. 기본 채팅 ---")
        message = "Python으로 간단한 웹 서버를 만드는 방법을 알려주세요."
        print(f"사용자: {message}")
        
        response = await client.get_response(message)
        print(f"AI: {response}")
        
        # 2. 다중 함수 도구 사용
        print("\n--- 2. 다중 함수 도구 사용 ---")
        message = "서울의 날씨와 현재 시간을 모두 알려주세요."
        print(f"사용자: {message}")
        
        # 여러 함수를 리스트로 전달
        tools = [get_weather, get_time_in_city]
        response = await client.get_response(message, tools=tools)
        print(f"AI: {response}")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print("💡 Azure OpenAI 리소스 설정을 확인해주세요.")

# 비동기 함수 실행
await demo_azure_openai_chat_client()

🚀 Azure OpenAI Chat Client 데모 시작...

--- 1. 기본 채팅 ---
사용자: Python으로 간단한 웹 서버를 만드는 방법을 알려주세요.
AI: 네, Python으로 아주 간단하게 웹 서버를 만드는 방법을 예로 들어 설명드릴게요.

### 1. 기본 HTTP 서버 (내장 모듈 이용)

Python의 표준 라이브러리만으로도 웹 서버를 시작할 수 있습니다.

```python
# Python 3.x 기준
from http.server import SimpleHTTPRequestHandler, HTTPServer

PORT = 8000

with HTTPServer(("localhost", PORT), SimpleHTTPRequestHandler) as httpd:
    print(f"Serving at http://localhost:{PORT}")
    httpd.serve_forever()
```

위 코드를 실행하면 `localhost:8000`에서 기본 파일 목록 또는 index.html 등을 제공합니다.

---

### 2. Flask 이용 (추천! 간단하고 확장 쉬움)

먼저 Flask를 설치합니다.

```bash
pip install flask
```

그리고 아래와 같이 작성합니다.

```python
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello():
    return 'Hello, World!'

if __name__ == '__main__':
    app.run(port=8000)
```

위 코드를 실행하면 `localhost:8000`에서 `Hello, World!`를 볼 수 있습니다.

---

### 요약

- **매우 간단** → `http.server` (기본 라이브러리)
- **기능 확장/실습용** → Flask와 같은 프레임워크 추천

궁금한 점이나 추가적으로 원하는 기능(파일 업로드, DB 연결 등)이 있으

## 7. 스트리밍 응답 예제

실시간으로 토큰이 생성되는 것을 확인할 수 있는 스트리밍 응답을 구현해봅시다.

In [7]:
async def demo_streaming_response():
    """스트리밍 응답을 시연합니다."""
    
    print("🚀 스트리밍 응답 데모 시작...")
    
    try:
        async with AzureAIAgentClient(async_credential=AsyncAzureCliCredential()) as client:
            
            print("\n--- 스트리밍 응답 (함수 호출 포함) ---")
            message = "도쿄와 런던의 날씨를 비교해서 상세히 설명해주세요."
            print(f"사용자: {message}")
            print("AI: ", end="", flush=True)
            
            # 스트리밍 응답 받기
            async for chunk in client.get_streaming_response(message, tools=get_weather):
                if str(chunk):
                    print(str(chunk), end="", flush=True)
            
            print("\n")  # 줄바꿈
            
            print("\n--- 일반 텍스트 스트리밍 ---")
            message = "인공지능의 미래에 대해 3문단으로 설명해주세요."
            print(f"사용자: {message}")
            print("AI: ", end="", flush=True)
            
            async for chunk in client.get_streaming_response(message):
                if str(chunk):
                    print(str(chunk), end="", flush=True)
            
            print("\n")  # 줄바꿈
            
    except Exception as e:
        print(f"❌ 오류 발생: {e}")

# 비동기 함수 실행
await demo_streaming_response()

🚀 스트리밍 응답 데모 시작...

--- 스트리밍 응답 (함수 호출 포함) ---
사용자: 도쿄와 런던의 날씨를 비교해서 상세히 설명해주세요.
AI: 네, 도쿄(Tokyo)와 런던(London)의 날씨를 비교해서 상세하게 설명드리겠습니다.

---

## 1. **기후 유형**

- **도쿄**:  
  - **기후 유형**: 온난 습윤 기후 (Cfa, 쾨펜 기후 구분)
  - **특징**: 사계절이 뚜렷하고, 여름이 덥고 습하며, 겨울은 비교적 온화함.

- **런던**:
  - **기후 유형**: 서안 해양성 기후 (Cfb, 쾨펜 기후 구분)
  - **특징**: 온화한 겨울과 서늘한 여름. 강수량이 고르게 분포되고, 흐린 날이 많고 습도가 높음.

---

## 2. **연평균 기온 및 강수량**

| 도시   | 연평균 기온 | 여름 최고 온도 | 겨울 최저 온도 | 연평균 강수량 | 특징      |
|-------|------------|---------------|---------------|---------------|-----------|
| **도쿄** | 약 15°C    | 30~35°C       | 2~8°C         | 1500mm        | 장마, 태풍 |
| **런던**| 약 11°C    | 20~25°C       | 2~7°C         | 600mm         | 흐린 날 많음|

---

## 3. **계절별 특징**

### **도쿄**
- **봄**: 3~5월, 온화함. 벚꽃이 피고 맑은 날씨가 많음.
- **여름**: 6~8월, 매우 덥고 습함. 6월엔 장마(우기), 8~9월엔 태풍이 자주 옴.
- **가을**: 9~11월, 시원함. 맑고 청량하며, 단풍철이 아름다움.
- **겨울**: 12~2월, 온화하지만 가끔 눈이 내림(약간).

### **런던**
- **봄**: 3~5월, 서늘하고 날씨 변화가 심함. 비가 자주 옴.
- **여름**: 6~8월, 비교적 시원함(한국·일본보다 덜 더움). 평

## 8. Azure OpenAI Assistants Client

`AzureOpenAIAssistantsClient`는 지속적인 대화와 컨텍스트 관리를 제공합니다.

In [8]:
async def demo_azure_assistants_client():
    """Azure OpenAI Assistants Client 사용법을 시연합니다."""
    
    print("🚀 Azure OpenAI Assistants Client 데모 시작...")
    
    try:
        async with AzureOpenAIAssistantsClient() as client:
            
            print("\n--- Assistant와의 대화 ---")
            
            # 첫 번째 메시지
            message1 = "안녕하세요! 저는 파이썬 개발자입니다. 오늘 뭔가 재미있는 프로젝트를 해보고 싶어요."
            print(f"사용자: {message1}")
            
            response1 = await client.get_response(message1, tools=get_weather)
            print(f"Assistant: {response1}")
            
            # 두 번째 메시지 (컨텍스트 유지)
            message2 = "그럼 현재 날씨에 맞는 프로젝트를 추천해주세요. 서울 날씨를 확인해서 결정해주세요."
            print(f"\n사용자: {message2}")
            
            response2 = await client.get_response(message2, tools=get_weather)
            print(f"Assistant: {response2}")
            
            print("\n💡 Assistant는 이전 대화 내용을 기억하고 있습니다!")
            
    except Exception as e:
        print(f"❌ 오류 발생: {e}")

# 비동기 함수 실행
await demo_azure_assistants_client()

🚀 Azure OpenAI Assistants Client 데모 시작...

--- Assistant와의 대화 ---
사용자: 안녕하세요! 저는 파이썬 개발자입니다. 오늘 뭔가 재미있는 프로젝트를 해보고 싶어요.
Assistant: 안녕하세요! 파이썬 개발자로서 재미있는 프로젝트를 찾으신다니 정말 멋집니다. 몇 가지 아이디어를 제안드릴게요. 관심 있는 분야나 목표에 따라 프로젝트를 추천해드릴 수 있습니다. 우선 간단하게 흥미로운 파이썬 프로젝트 아이디어를 몇 가지 소개해드릴게요:

1. **날씨 정보 챗봇**  
   - 사용자가 위치를 입력하면, 실시간 날씨 정보를 웹에서 가져와 제공합니다.

2. **이미지 합성/필터 앱 (OpenCV 활용)**  
   - 여러 이미지를 합성하거나 간단한 필터를 적용하는 이미처리 프로그램입니다.

3. **미니게임 만들기**  
   - Python의 `pygame`을 활용해서 간단한 2D 게임(예: 테트리스, 뱀 게임) 제작.

4. **웹 크롤러 & 데이터 분석**  
   - 특정 웹사이트에서 데이터를 수집하고, Pandas로 시각화 및 분석.

5. **음성 인식 & 텍스트 변환**  
   - 마이크 입력을 받아 음성을 텍스트로 변환.

혹시 관심 있는 주제나 도전해보고 싶은 분야가 있으신가요? 아니면 위의 아이템 중 끌리는 것이 있으신가요? 여러분의 흥미와 실력에 맞춰 구체적인 프로젝트 예제와 진행방법도 더 안내드릴 수 있어요!

사용자: 그럼 현재 날씨에 맞는 프로젝트를 추천해주세요. 서울 날씨를 확인해서 결정해주세요.
Assistant: 오늘 서울은 흐린 날씨에 최고 기온이 23°C입니다.

이런 날씨에는 다음과 같은 프로젝트를 추천합니다:

1. 실내 활동 프로젝트
   - DIY 공예, 그림 그리기, 캔들 만들기 등 집에서 할 수 있는 창작 활동.
   - 온라인 강의나 원데이 클래스 참여(요리, 악기 등).

2. 독서 또는 글쓰기
   - 카페나 도서관에서 좋아하는 책 읽기.
   - 일기나 에세이,

## 9. 구조화된 응답 (Azure Responses Client)

`AzureOpenAIResponsesClient`를 사용하여 JSON 형태의 구조화된 응답을 받아봅시다.

In [9]:
async def demo_azure_responses_client():
    """Azure OpenAI Responses Client 사용법을 시연합니다."""
    
    print("🚀 Azure OpenAI Responses Client 데모 시작...")
    
    try:
        client = AzureOpenAIResponsesClient()
        
        print("\n--- 구조화된 응답 (비스트리밍) ---")
        message = "부산의 날씨를 조회해주세요."
        print(f"사용자: {message}")
        
        # 구조화된 응답 요청
        response = await client.get_response(
            message,
            tools=get_weather,
            response_format=WeatherInfo
        )
        
        print(f"응답 타입: {type(response.value)}")
        print(f"구조화된 응답: {response.value}")
        
        # 개별 필드 접근
        if hasattr(response.value, 'location'):
            print(f"\n📍 위치: {response.value.location}")
            print(f"🌤️  날씨: {response.value.condition}")
            print(f"🌡️  온도: {response.value.temperature}°C")
            print(f"📝 설명: {response.value.description}")
        
        print("\n--- 구조화된 응답 (스트리밍) ---")
        message = "제주도의 날씨 정보를 구조화된 형태로 알려주세요."
        print(f"사용자: {message}")
        
        # 스트리밍으로 구조화된 응답 받기
        response = await ChatResponse.from_chat_response_generator(
            client.get_streaming_response(message, tools=get_weather, response_format=WeatherInfo),
            output_format_type=WeatherInfo,
        )
        
        print(f"스트리밍 구조화된 응답: {response.value}")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")

# 비동기 함수 실행
await demo_azure_responses_client()

🚀 Azure OpenAI Responses Client 데모 시작...

--- 구조화된 응답 (비스트리밍) ---
사용자: 부산의 날씨를 조회해주세요.
❌ 오류 발생: <class 'agent_framework.azure._responses_client.AzureOpenAIResponsesClient'> service failed to complete the prompt: Error code: 400 - {'error': {'code': 'BadRequest', 'message': 'API version not supported'}}


## 10. 응답 취소 메커니즘

장시간 실행되는 요청을 중간에 취소하는 방법을 알아봅시다.

In [10]:
async def demo_response_cancellation():
    """응답 취소 메커니즘을 시연합니다."""
    
    print("🚀 응답 취소 메커니즘 데모 시작...")
    
    try:
        async with AzureAIAgentClient(async_credential=AsyncAzureCliCredential()) as client:
            
            print("\n--- 시나리오 1: 타임아웃으로 요청 취소 ---")
            
            # 긴 응답을 요청하는 메시지
            message = "인공지능의 역사부터 현재까지의 발전 과정을 매우 상세하게 설명해주세요. 각 시대별로 주요 인물과 기술적 성과를 포함해주세요."
            print(f"사용자: {message}")
            print("⏰ 2초 후 요청을 취소합니다...")
            
            try:
                # 응답 요청을 태스크로 생성
                task = asyncio.create_task(
                    client.get_response(message)
                )
                
                # 2초 대기 후 취소
                await asyncio.sleep(2)
                print("⏹️  요청을 취소합니다...")
                task.cancel()
                
                # 취소된 태스크 대기
                await task
                
            except asyncio.CancelledError:
                print("✅ 요청이 성공적으로 취소되었습니다!")
            
            print("\n--- 시나리오 2: 스트리밍 응답 취소 ---")
            
            message = "세계 각국의 문화와 전통에 대해 매우 길고 상세한 설명을 해주세요."
            print(f"사용자: {message}")
            print("⏰ 스트리밍 중 3초 후 취소합니다...")
            
            try:
                task = asyncio.create_task(
                    stream_with_cancellation(client, message)
                )
                
                await task
                
            except asyncio.CancelledError:
                print("\n✅ 스트리밍 응답이 성공적으로 취소되었습니다!")
    
    except Exception as e:
        print(f"❌ 오류 발생: {e}")

async def stream_with_cancellation(client, message):
    """스트리밍 응답을 받으면서 중간에 취소하는 함수"""
    print("AI: ", end="", flush=True)
    
    start_time = asyncio.get_event_loop().time()
    async for chunk in client.get_streaming_response(message):
        if str(chunk):
            print(str(chunk), end="", flush=True)
        
        # 3초 후 취소
        current_time = asyncio.get_event_loop().time()
        if current_time - start_time > 3:
            print("\n⏹️  스트리밍을 중단합니다...")
            raise asyncio.CancelledError()

# 비동기 함수 실행
await demo_response_cancellation()

🚀 응답 취소 메커니즘 데모 시작...

--- 시나리오 1: 타임아웃으로 요청 취소 ---
사용자: 인공지능의 역사부터 현재까지의 발전 과정을 매우 상세하게 설명해주세요. 각 시대별로 주요 인물과 기술적 성과를 포함해주세요.
⏰ 2초 후 요청을 취소합니다...
⏹️  요청을 취소합니다...
✅ 요청이 성공적으로 취소되었습니다!

--- 시나리오 2: 스트리밍 응답 취소 ---
사용자: 세계 각국의 문화와 전통에 대해 매우 길고 상세한 설명을 해주세요.
⏰ 스트리밍 중 3초 후 취소합니다...
AI: 

Failed to detach context
Traceback (most recent call last):
  File "/Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/opentelemetry/trace/__init__.py", line 589, in use_span
    yield span
  File "/Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/observability.py", line 1342, in _get_span
    yield current_span
  File "/Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/observability.py", line 963, in trace_get_streaming_response
    yield update
GeneratorExit

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/opentelemetry/context/__init__.py", line 155, in detach
    _RUNTIME_CONTEXT.detach(token)
  File "/Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
    self._current_context.reset(


⏹️  스트리밍을 중단합니다...

✅ 스트리밍 응답이 성공적으로 취소되었습니다!


## 11. 에러 처리 및 모범 사례

실제 프로덕션 환경에서 사용할 때의 에러 처리 패턴과 모범 사례를 알아봅시다.

In [11]:
async def demo_error_handling_and_best_practices():
    """에러 처리와 모범 사례를 시연합니다."""
    
    print("🚀 에러 처리 및 모범 사례 데모 시작...")
    
    # 1. 연결 재시도 로직
    print("\n--- 1. 재시도 로직 ---")
    
    async def chat_with_retry(client, message, max_retries=3):
        """재시도 로직이 포함된 채팅 함수"""
        for attempt in range(max_retries):
            try:
                print(f"🔄 시도 {attempt + 1}/{max_retries}")
                response = await client.get_response(message)
                return response
            except Exception as e:
                print(f"❌ 시도 {attempt + 1} 실패: {e}")
                if attempt == max_retries - 1:
                    raise
                await asyncio.sleep(1 * (attempt + 1))  # 지수 백오프
    
    # 2. 컨텍스트 매니저 사용 (리소스 정리)
    print("\n--- 2. 적절한 리소스 관리 ---")
    
    try:
        async with AzureAIAgentClient(async_credential=AsyncAzureCliCredential()) as client:
            message = "간단한 인사 메시지를 보내주세요."
            print(f"사용자: {message}")
            
            response = await chat_with_retry(client, message)
            print(f"AI: {response}")
            print("✅ 리소스가 자동으로 정리됩니다.")
            
    except Exception as e:
        print(f"❌ 최종 실패: {e}")
    
    # 3. 함수 도구 에러 처리
    print("\n--- 3. 함수 도구 에러 처리 ---")
    
    def robust_get_weather(
        location: Annotated[str, Field(description="날씨를 조회할 도시 이름")],
    ) -> str:
        """에러 처리가 포함된 날씨 조회 함수"""
        try:
            if not location or location.strip() == "":
                return "❌ 유효한 도시 이름을 입력해주세요."
            
            # 실제 API 호출 시뮬레이션
            conditions = ["맑음", "흐림", "비", "눈"]
            temperature = randint(10, 30)
            condition = conditions[randint(0, len(conditions)-1)]
            
            return f"✅ {location}의 날씨: {condition}, 기온: {temperature}°C"
            
        except Exception as e:
            return f"❌ 날씨 조회 중 오류 발생: {str(e)}"
    
    # 에러 처리가 포함된 함수로 테스트
    try:
        async with AzureAIAgentClient(async_credential=AsyncAzureCliCredential()) as client:
            message = "잘못된 도시명과 정상적인 도시명 모두로 날씨를 조회해주세요: '', '서울'"
            print(f"사용자: {message}")
            
            response = await client.get_response(message, tools=robust_get_weather)
            print(f"AI: {response}")
            
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
    
    # 4. 모범 사례 요약
    print("\n--- 📋 모범 사례 요약 ---")
    best_practices = [
        "1. 항상 async with 문을 사용하여 리소스를 자동으로 정리하세요",
        "2. 재시도 로직을 구현하여 일시적인 네트워크 오류에 대응하세요",
        "3. 함수 도구에서는 입력 검증과 에러 처리를 철저히 하세요",
        "4. 환경 변수를 사용하여 민감한 정보를 안전하게 관리하세요",
        "5. 스트리밍 응답 시 적절한 취소 메커니즘을 구현하세요",
        "6. 로깅을 통해 디버깅과 모니터링을 용이하게 하세요"
    ]
    
    for practice in best_practices:
        print(f"✅ {practice}")

# 비동기 함수 실행
await demo_error_handling_and_best_practices()

Unclosed connection
client_connection: Connection<ConnectionKey(host='aoai-q4u67epqs2xo4.services.ai.azure.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=None)>


🚀 에러 처리 및 모범 사례 데모 시작...

--- 1. 재시도 로직 ---

--- 2. 적절한 리소스 관리 ---
사용자: 간단한 인사 메시지를 보내주세요.
🔄 시도 1/3
AI: 안녕하세요!  
오늘도 좋은 하루 보내세요 :)
✅ 리소스가 자동으로 정리됩니다.

--- 3. 함수 도구 에러 처리 ---
사용자: 잘못된 도시명과 정상적인 도시명 모두로 날씨를 조회해주세요: '', '서울'
AI: 결과를 안내드립니다:

- '' (빈 도시명): ❌ 유효한 도시 이름을 입력해주세요.
- '서울': ✅ 서울의 날씨는 "비"이고, 현재 기온은 22°C입니다.

더 조회할 도시가 있으시면 말씀해 주세요!

--- 📋 모범 사례 요약 ---
✅ 1. 항상 async with 문을 사용하여 리소스를 자동으로 정리하세요
✅ 2. 재시도 로직을 구현하여 일시적인 네트워크 오류에 대응하세요
✅ 3. 함수 도구에서는 입력 검증과 에러 처리를 철저히 하세요
✅ 4. 환경 변수를 사용하여 민감한 정보를 안전하게 관리하세요
✅ 5. 스트리밍 응답 시 적절한 취소 메커니즘을 구현하세요
✅ 6. 로깅을 통해 디버깅과 모니터링을 용이하게 하세요


## 12. 종합 실습

지금까지 학습한 내용을 바탕으로 실제 애플리케이션에서 사용할 수 있는 통합 예제를 만들어봅시다.

In [12]:
class WeatherAssistant:
    """종합적인 날씨 어시스턴트 클래스"""
    
    def __init__(self):
        self.client = None
        self.conversation_history = []
    
    async def __aenter__(self):
        self.client = AzureAIAgentClient(async_credential=AsyncAzureCliCredential())
        await self.client.__aenter__()
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        if self.client:
            await self.client.__aexit__(exc_type, exc_val, exc_tb)
    
    def get_weather_extended(
        self,
        location: Annotated[str, Field(description="날씨를 조회할 도시 이름")]
    ) -> str:
        """확장된 날씨 조회 함수"""
        try:
            conditions = ["맑음", "흐림", "비", "눈", "안개"]
            temperature = randint(-5, 35)
            condition = conditions[randint(0, len(conditions)-1)]
            humidity = randint(30, 90)
            wind_speed = randint(0, 20)
            
            weather_info = {
                "location": location,
                "condition": condition,
                "temperature": temperature,
                "humidity": humidity,
                "wind_speed": wind_speed
            }
            
            self.conversation_history.append(f"날씨 조회: {location}")
            
            return (f"{location}의 현재 날씨:\n"
                   f"🌤️ 날씨: {condition}\n"
                   f"🌡️ 기온: {temperature}°C\n"
                   f"💧 습도: {humidity}%\n"
                   f"💨 풍속: {wind_speed}km/h")
        
        except Exception as e:
            return f"❌ {location}의 날씨 조회 실패: {str(e)}"
    
    async def chat(self, message: str, use_streaming: bool = False) -> str:
        """채팅 인터페이스"""
        try:
            self.conversation_history.append(f"사용자: {message}")
            
            if use_streaming:
                print("AI: ", end="", flush=True)
                full_response = ""
                async for chunk in self.client.get_streaming_response(
                    message,
                    tools=self.get_weather_extended
                ):
                    if str(chunk):
                        print(str(chunk), end="", flush=True)
                        full_response += str(chunk)
                print()  # 줄바꿈
                return full_response
            else:
                response = await self.client.get_response(
                    message,
                    tools=self.get_weather_extended
                )
                return str(response)
        
        except Exception as e:
            return f"❌ 오류 발생: {str(e)}"
    
    def get_conversation_summary(self) -> str:
        """대화 요약 반환"""
        return "\n".join(self.conversation_history[-5:])  # 최근 5개 항목

# 종합 실습 실행
async def comprehensive_demo():
    """종합 실습 데모"""
    
    print("🎓 Azure AI Agent Framework 종합 실습")
    print("=" * 50)
    
    async with WeatherAssistant() as assistant:
        
        # 시나리오 1: 기본 날씨 조회
        print("\n📍 시나리오 1: 기본 날씨 조회")
        message1 = "오늘 서울 날씨가 어때요?"
        print(f"사용자: {message1}")
        response1 = await assistant.chat(message1)
        print(f"AI: {response1}")
        
        # 시나리오 2: 다중 도시 비교 (스트리밍)
        print(f"\n🌍 시나리오 2: 다중 도시 날씨 비교 (스트리밍)")
        message2 = "서울, 도쿄, 뉴욕의 날씨를 비교해서 여행하기 좋은 곳을 추천해주세요."
        print(f"사용자: {message2}")
        await assistant.chat(message2, use_streaming=True)
        
        # 시나리오 3: 컨텍스트 기반 질문
        print(f"\n🔗 시나리오 3: 컨텍스트 기반 대화")
        message3 = "그럼 그 중에서 가장 따뜻한 곳은 어디인가요?"
        print(f"사용자: {message3}")
        response3 = await assistant.chat(message3)
        print(f"AI: {response3}")
        
        # 대화 요약 출력
        print(f"\n📋 대화 요약:")
        print(assistant.get_conversation_summary())

# 종합 실습 실행
await comprehensive_demo()

🎓 Azure AI Agent Framework 종합 실습

📍 시나리오 1: 기본 날씨 조회
사용자: 오늘 서울 날씨가 어때요?
AI: 오늘 서울의 날씨는 눈이 내리고 있으며, 기온은 0°C입니다. 습도는 72%, 풍속은 19km/h입니다. 외출 시 따뜻하게 입으시고, 눈길에 주의하세요!

🌍 시나리오 2: 다중 도시 날씨 비교 (스트리밍)
사용자: 서울, 도쿄, 뉴욕의 날씨를 비교해서 여행하기 좋은 곳을 추천해주세요.
AI: 현재(2024년 6월 기준) 서울, 도쿄, 뉴욕의 평균적인 날씨와 여행 적합성을 비교해서 추천드리겠습니다.

---

### 1. 서울
- **기온**: 평균 21~28°C (초여름, 종종 무더위)
- **날씨**: 장마철 시작(6월 중·하순부터), 비가 자주 오고 습함
- **특징**: 갑자기 더워질 수 있으며, 습도 높음  
- **여행 팁**: 비를 대비해 우산이나 우비 필수, 실내 관광지 추천

---

### 2. 도쿄
- **기온**: 평균 20~27°C
- **날씨**: 일본 역시 장마 시즌으로 비가 잦음, 습함  
- **특징**: 서울과 비슷하게 습도가 높고 비가 많음
- **여행 팁**: 우천 시 방문할 수 있는 박물관, 쇼핑몰, 카페 등 추천

---

### 3. 뉴욕
- **기온**: 평균 18~27°C
- **날씨**: 맑은 날이 많고, 상대적으로 습도가 낮음  
- **특징**: 쾌적한 초여름 날씨, 야외 활동에 적합
- **여행 팁**: 센트럴 파크, 브루클린, 야외 페스티벌 등 야외를 즐기기 좋음

---

## 추천

**장마철인 서울과 도쿄는 습하고 비가 많이 오는 반면**,  
**뉴욕은 맑고 쾌적한 초여름 날씨**로 여행하기 훨씬 좋은 시즌입니다.

**따라서, 6월에는 뉴욕을 추천**합니다!  
야외 관광, 공원 산책, 도시 투어 등 날씨에 방해받지 않고 다양한 여행을 즐길 수 있습니다.

필요하다면 각 도시의 실시간 상세 기상정보와 이벤트/축제 정보도 알려드릴 수 있습니다 :)

🔗 시나리오 3

## 13. 마무리 및 추가 학습 자료

🎉 **축하합니다!** Azure AI Agent Framework의 핵심 기능들을 모두 학습하셨습니다.

### 학습한 내용 요약:

1. ✅ **uv 환경 설정** - 현대적인 Python 패키지 관리 (Python 3.12)
2. ✅ **Azure AI Agent Client** - 기본적인 AI 상호작용
3. ✅ **Azure OpenAI Chat Client** - OpenAI 모델 직접 사용
4. ✅ **Function Calling** - 외부 함수와 AI 연동
5. ✅ **스트리밍 응답** - 실시간 토큰 생성
6. ✅ **Assistants Client** - 컨텍스트 유지 대화
7. ✅ **구조화된 응답** - JSON 형태 출력
8. ✅ **응답 취소** - 타임아웃 및 취소 메커니즘
9. ✅ **에러 처리** - 프로덕션 환경 모범 사례
10. ✅ **종합 실습** - 실제 애플리케이션 구현

### uv 환경 관리 명령어 (Python 3.12):

```bash
# Python 3.12로 새 가상환경 생성
uv venv --python 3.12

# 의존성 설치
uv sync --dev

# 새 패키지 추가
uv add package-name

# 개발 의존성 추가  
uv add --dev package-name

# 패키지 업데이트
uv sync --upgrade

# 가상환경 활성화
source .venv/bin/activate  # macOS/Linux
# 또는
.venv\Scripts\activate     # Windows

# Python 버전 확인
uv run python --version  # Python 3.12.x

# Jupyter 노트북 실행
uv run jupyter notebook
```

### 다음 단계:

- 📚 [Agent Framework 공식 문서](https://github.com/microsoft/agent-framework) 
- 🔧 더 복잡한 Function Calling 구현해보기
- 🌐 웹 애플리케이션에 통합하기
- 📊 로깅 및 모니터링 시스템 구축
- 🧪 다양한 AI 모델 실험해보기
- ⚡ [uv 공식 문서](https://docs.astral.sh/uv/) - 현대적인 Python 패키지 관리
- 🐍 Python 3.12의 새로운 기능 활용

### 추가 실습 아이디어:

1. **멀티모달 입력** - 이미지와 텍스트를 함께 처리
2. **대화 메모리** - 장기간 대화 기록 저장
3. **외부 API 연동** - 실제 날씨, 뉴스 API 사용
4. **사용자 인터페이스** - Streamlit, FastAPI 웹앱 구축
5. **uv 스크립트** - pyproject.toml의 [tool.uv.scripts] 활용
6. **Python 3.12 활용** - 새로운 타입 힌트와 성능 개선 기능 사용

## 14. 실습 과제 (Homework)

이제 배운 내용을 직접 적용해볼 시간입니다! 아래 과제들을 완성해보세요.

### 📝 과제 개요

각 과제는 노트북에서 학습한 내용을 바탕으로 구성되어 있습니다. 코드 셀에 있는 `# TODO` 주석을 찾아 구현해주세요.

**난이도:**
- ⭐ 기초: 노트북 예제를 참고하면 쉽게 해결 가능
- ⭐⭐ 중급: 여러 개념을 조합해야 함
- ⭐⭐⭐ 고급: 창의적인 해결 방법이 필요

### 과제 목록

1. **⭐ 과제 1**: 사용자 정의 함수 도구 만들기
2. **⭐⭐ 과제 2**: 스트리밍 응답으로 실시간 번역기 만들기
3. **⭐⭐ 과제 3**: 구조화된 응답으로 여행 계획 생성하기
4. **⭐⭐⭐ 과제 4**: 대화형 AI 어시스턴트 만들기

### ⭐ 과제 1: 사용자 정의 함수 도구 만들기

**목표**: Function Calling을 활용하여 새로운 함수 도구들을 만들어봅니다.

**요구사항**:
1. `get_exchange_rate()` 함수를 구현하세요 (통화 변환)
2. `calculate_tip()` 함수를 구현하세요 (팁 계산)
3. 두 함수를 AI와 연동하여 사용자 질문에 답하세요

**힌트**: 
- 노트북의 `get_weather()` 함수 구조를 참고하세요
- `Annotated`와 `Field`를 사용하여 매개변수를 명확히 설명하세요

In [13]:
### 과제 1 구현 ###

# TODO: 환율 조회 함수를 구현하세요
def get_exchange_rate(
    from_currency: Annotated[str, Field(description="출발 통화 (예: USD, KRW)")],
    to_currency: Annotated[str, Field(description="도착 통화 (예: USD, KRW)")],
    amount: Annotated[float, Field(description="변환할 금액")] = 1.0
) -> str:
    """
    통화를 변환합니다 (시뮬레이션).
    
    Args:
        from_currency: 출발 통화 코드
        to_currency: 도착 통화 코드
        amount: 변환할 금액
    
    Returns:
        변환 결과 문자열
    """
    # TODO: 여기에 코드를 작성하세요
    # 힌트: 간단한 환율 사전을 만들거나 임의의 환율을 생성하세요
    pass


# TODO: 팁 계산 함수를 구현하세요
def calculate_tip(
    bill_amount: Annotated[float, Field(description="총 계산서 금액")],
    tip_percentage: Annotated[int, Field(description="팁 비율 (10, 15, 20 등)")] = 15
) -> str:
    """
    계산서 금액에 대한 팁을 계산합니다.
    
    Args:
        bill_amount: 총 계산서 금액
        tip_percentage: 팁 비율 (기본값: 15%)
    
    Returns:
        팁 금액과 총 금액 정보
    """
    # TODO: 여기에 코드를 작성하세요
    # 힌트: 팁 금액과 총 금액(계산서 + 팁)을 계산하여 반환하세요
    pass


# TODO: 구현한 함수들을 테스트하세요
async def test_homework_1():
    """과제 1을 테스트합니다."""
    
    print("🧪 과제 1 테스트 시작...\n")
    
    try:
        async with AzureAIAgentClient(async_credential=AsyncAzureCliCredential()) as client:
            
            # 테스트 1: 환율 조회
            print("--- 테스트 1: 환율 조회 ---")
            message1 = "100 달러를 한국 원화로 변환하면 얼마인가요?"
            print(f"사용자: {message1}")
            
            # TODO: get_exchange_rate 함수를 tools로 전달하세요
            response1 = await client.get_response(message1, tools=None)  # 여기를 수정하세요
            print(f"AI: {response1}\n")
            
            # 테스트 2: 팁 계산
            print("--- 테스트 2: 팁 계산 ---")
            message2 = "식사 비용이 50,000원인데 20% 팁을 주면 총 얼마인가요?"
            print(f"사용자: {message2}")
            
            # TODO: calculate_tip 함수를 tools로 전달하세요
            response2 = await client.get_response(message2, tools=None)  # 여기를 수정하세요
            print(f"AI: {response2}\n")
            
            # 테스트 3: 두 함수 모두 사용
            print("--- 테스트 3: 복합 질문 ---")
            message3 = "미국에서 30달러 식사를 하고 15% 팁을 주면 한국 돈으로 총 얼마인가요?"
            print(f"사용자: {message3}")
            
            # TODO: 두 함수를 모두 tools로 전달하세요
            response3 = await client.get_response(message3, tools=None)  # 여기를 수정하세요
            print(f"AI: {response3}\n")
            
            print("✅ 과제 1 테스트 완료!")
            
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print("💡 함수 구현과 tools 매개변수를 확인해주세요.")

# 테스트 실행
# await test_homework_1()  # 주석을 해제하고 실행하세요

### ⭐⭐ 과제 2: 스트리밍 응답으로 실시간 번역기 만들기

**목표**: 스트리밍 응답을 활용하여 실시간으로 번역 결과를 보여주는 번역기를 만듭니다.

**요구사항**:
1. `translate_text()` 함수를 구현하세요
2. 스트리밍 방식으로 번역 결과를 출력하세요
3. 여러 언어 간 번역을 지원하세요

**힌트**:
- 노트북의 `demo_streaming_response()` 예제를 참고하세요
- `get_streaming_response()` 메서드를 사용하세요

In [14]:
### 과제 2 구현 ###

# TODO: 번역 함수를 구현하세요
def translate_text(
    text: Annotated[str, Field(description="번역할 텍스트")],
    source_language: Annotated[str, Field(description="원본 언어 (예: 영어, 한국어, 일본어)")],
    target_language: Annotated[str, Field(description="목표 언어 (예: 영어, 한국어, 일본어)")]
) -> str:
    """
    텍스트를 다른 언어로 번역합니다 (시뮬레이션).
    
    Args:
        text: 번역할 텍스트
        source_language: 원본 언어
        target_language: 목표 언어
    
    Returns:
        번역 정보 (실제 번역은 AI가 수행)
    """
    # TODO: 여기에 코드를 작성하세요
    # 힌트: 번역 요청 정보를 반환하세요. 실제 번역은 AI가 수행합니다.
    pass


# TODO: 스트리밍 번역기 클래스를 구현하세요
class StreamingTranslator:
    """실시간 번역기 클래스"""
    
    def __init__(self):
        self.client = None
    
    async def __aenter__(self):
        # TODO: AzureAIAgentClient를 생성하세요
        pass
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        # TODO: 클라이언트를 정리하세요
        pass
    
    async def translate_streaming(
        self, 
        text: str, 
        source_lang: str = "한국어",
        target_lang: str = "영어"
    ):
        """
        스트리밍 방식으로 번역을 수행합니다.
        
        Args:
            text: 번역할 텍스트
            source_lang: 원본 언어
            target_lang: 목표 언어
        """
        # TODO: 여기에 코드를 작성하세요
        # 힌트: 
        # 1. 적절한 프롬프트를 만드세요
        # 2. get_streaming_response()를 사용하여 스트리밍으로 결과를 받으세요
        # 3. 각 청크를 실시간으로 출력하세요
        pass


# TODO: 번역기를 테스트하세요
async def test_homework_2():
    """과제 2를 테스트합니다."""
    
    print("🧪 과제 2 테스트 시작...\n")
    
    try:
        # TODO: StreamingTranslator를 사용하여 테스트하세요
        
        # 테스트 1: 한국어 -> 영어
        print("--- 테스트 1: 한국어 -> 영어 (스트리밍) ---")
        text1 = "안녕하세요! Azure AI Agent Framework를 사용하면 강력한 AI 애플리케이션을 쉽게 만들 수 있습니다."
        print(f"원본 텍스트: {text1}")
        print("번역 결과: ", end="", flush=True)
        # TODO: translate_streaming()을 호출하세요
        
        print("\n")
        
        # 테스트 2: 영어 -> 일본어
        print("--- 테스트 2: 영어 -> 일본어 (스트리밍) ---")
        text2 = "Artificial Intelligence is transforming the way we work and live."
        print(f"원본 텍스트: {text2}")
        print("번역 결과: ", end="", flush=True)
        # TODO: translate_streaming()을 호출하세요
        
        print("\n")
        
        print("✅ 과제 2 테스트 완료!")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print("💡 StreamingTranslator 클래스 구현을 확인해주세요.")

# 테스트 실행
# await test_homework_2()  # 주석을 해제하고 실행하세요

### ⭐⭐ 과제 3: 구조화된 응답으로 여행 계획 생성하기

**목표**: Pydantic 모델을 사용하여 구조화된 여행 계획을 생성합니다.

**요구사항**:
1. `TravelPlan` Pydantic 모델을 정의하세요
2. 여행지 정보를 조회하는 함수를 만드세요
3. 구조화된 응답으로 여행 계획을 받아 출력하세요

**힌트**:
- 노트북의 `WeatherInfo` 모델과 `demo_azure_responses_client()` 예제를 참고하세요
- `AzureOpenAIResponsesClient`와 `response_format` 매개변수를 사용하세요

In [15]:
### 과제 3 구현 ###

from typing import List

# TODO: 여행 계획 Pydantic 모델을 정의하세요
class TravelPlan(BaseModel):
    """여행 계획의 구조화된 모델"""
    # TODO: 필요한 필드들을 정의하세요
    # 힌트: destination, duration_days, activities, budget, best_season 등
    pass


# TODO: 여행지 정보 조회 함수를 구현하세요
def get_destination_info(
    destination: Annotated[str, Field(description="여행지 이름")],
) -> str:
    """
    여행지의 주요 정보를 반환합니다 (시뮬레이션).
    
    Args:
        destination: 여행지 이름
    
    Returns:
        여행지 정보
    """
    # TODO: 여기에 코드를 작성하세요
    # 힌트: 여행지별 특징, 인기 활동, 추천 시즌 등의 정보를 반환하세요
    pass


# TODO: 여행 계획 생성기를 테스트하세요
async def test_homework_3():
    """과제 3을 테스트합니다."""
    
    print("🧪 과제 3 테스트 시작...\n")
    
    try:
        # TODO: AzureOpenAIResponsesClient를 생성하세요
        client = None  # 여기를 수정하세요
        
        # 테스트 1: 구조화된 여행 계획 생성
        print("--- 테스트 1: 도쿄 여행 계획 (구조화된 응답) ---")
        message1 = "일본 도쿄로 3박 4일 여행을 계획하고 있어요. 여행 계획을 만들어주세요."
        print(f"사용자: {message1}\n")
        
        # TODO: get_response()를 호출하여 구조화된 응답을 받으세요
        # 힌트: response_format=TravelPlan을 사용하세요
        response1 = None  # 여기를 수정하세요
        
        # TODO: 응답을 파싱하고 출력하세요
        if response1:
            print(f"응답 타입: {type(response1.value)}")
            print(f"\n📋 여행 계획:")
            # TODO: TravelPlan의 각 필드를 출력하세요
            pass
        
        print("\n")
        
        # 테스트 2: 다른 여행지
        print("--- 테스트 2: 파리 여행 계획 (함수 호출 포함) ---")
        message2 = "프랑스 파리로 일주일 여행을 가려고 해요. 상세한 계획을 짜주세요."
        print(f"사용자: {message2}\n")
        
        # TODO: get_destination_info 함수를 tools로 전달하고 구조화된 응답을 받으세요
        response2 = None  # 여기를 수정하세요
        
        if response2:
            print(f"\n📋 여행 계획:")
            # TODO: TravelPlan의 각 필드를 출력하세요
            pass
        
        print("\n✅ 과제 3 테스트 완료!")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print("💡 TravelPlan 모델 정의와 response_format 사용을 확인해주세요.")

# 테스트 실행
# await test_homework_3()  # 주석을 해제하고 실행하세요

### ⭐⭐⭐ 과제 4: 대화형 AI 어시스턴트 만들기

**목표**: 지금까지 배운 모든 기능을 통합하여 완전한 대화형 AI 어시스턴트를 만듭니다.

**요구사항**:
1. 여러 함수 도구를 지원하는 어시스턴트 클래스를 만드세요
2. 대화 히스토리를 관리하세요
3. 스트리밍과 비스트리밍 모드를 선택할 수 있게 하세요
4. 에러 처리와 재시도 로직을 구현하세요
5. 대화 요약 기능을 추가하세요

**힌트**:
- 노트북의 `WeatherAssistant` 클래스를 참고하세요
- 모든 학습 내용을 종합적으로 활용하세요

In [16]:
### 과제 4 구현 ###

# TODO: 종합 AI 어시스턴트 클래스를 구현하세요
class ComprehensiveAssistant:
    """종합적인 AI 어시스턴트 클래스"""
    
    def __init__(self, name: str = "AI Assistant"):
        """
        어시스턴트를 초기화합니다.
        
        Args:
            name: 어시스턴트 이름
        """
        # TODO: 필요한 속성들을 초기화하세요
        # 힌트: client, conversation_history, name 등
        pass
    
    async def __aenter__(self):
        """컨텍스트 매니저 진입"""
        # TODO: AzureAIAgentClient를 생성하고 초기화하세요
        pass
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """컨텍스트 매니저 종료"""
        # TODO: 클라이언트를 정리하세요
        pass
    
    # TODO: 유용한 함수 도구들을 메서드로 추가하세요
    def get_information(
        self,
        topic: Annotated[str, Field(description="정보를 조회할 주제")],
        category: Annotated[str, Field(description="카테고리 (기술, 여행, 요리 등)")] = "일반"
    ) -> str:
        """주제에 대한 정보를 반환합니다."""
        # TODO: 구현하세요
        pass
    
    def calculate(
        self,
        expression: Annotated[str, Field(description="계산할 수식")],
    ) -> str:
        """수식을 계산합니다."""
        # TODO: 구현하세요 (eval 사용 시 보안에 주의)
        pass
    
    async def chat(
        self, 
        message: str, 
        use_streaming: bool = False,
        max_retries: int = 3
    ) -> str:
        """
        사용자 메시지에 응답합니다.
        
        Args:
            message: 사용자 메시지
            use_streaming: 스트리밍 모드 사용 여부
            max_retries: 최대 재시도 횟수
        
        Returns:
            AI 응답
        """
        # TODO: 여기에 코드를 작성하세요
        # 힌트:
        # 1. 재시도 로직을 구현하세요
        # 2. 대화 히스토리에 메시지를 추가하세요
        # 3. 스트리밍/비스트리밍 모드를 지원하세요
        # 4. 함수 도구들을 tools로 전달하세요
        pass
    
    def get_conversation_history(self, last_n: int = 10) -> List[str]:
        """
        최근 대화 히스토리를 반환합니다.
        
        Args:
            last_n: 반환할 최근 대화 수
        
        Returns:
            대화 히스토리 리스트
        """
        # TODO: 구현하세요
        pass
    
    def clear_history(self):
        """대화 히스토리를 초기화합니다."""
        # TODO: 구현하세요
        pass
    
    async def get_summary(self) -> str:
        """
        지금까지의 대화를 요약합니다.
        
        Returns:
            대화 요약
        """
        # TODO: 구현하세요
        # 힌트: 대화 히스토리를 AI에게 전달하여 요약을 요청하세요
        pass


# TODO: 어시스턴트를 테스트하세요
async def test_homework_4():
    """과제 4를 테스트합니다."""
    
    print("🧪 과제 4 테스트 시작...\n")
    print("=" * 60)
    
    try:
        # TODO: ComprehensiveAssistant를 사용하여 대화를 진행하세요
        async with ComprehensiveAssistant(name="MyAI") as assistant:
            
            # 시나리오 1: 기본 대화
            print("\n📍 시나리오 1: 기본 정보 질문")
            message1 = "파이썬의 주요 특징을 3가지만 알려주세요."
            print(f"사용자: {message1}")
            response1 = await assistant.chat(message1)
            print(f"AI: {response1}")
            
            # 시나리오 2: 함수 도구 사용
            print(f"\n🔧 시나리오 2: 계산 기능 사용")
            message2 = "15 곱하기 23을 계산해주세요."
            print(f"사용자: {message2}")
            response2 = await assistant.chat(message2)
            print(f"AI: {response2}")
            
            # 시나리오 3: 스트리밍 모드
            print(f"\n🌊 시나리오 3: 스트리밍 모드로 응답")
            message3 = "인공지능의 미래에 대해 간단히 설명해주세요."
            print(f"사용자: {message3}")
            await assistant.chat(message3, use_streaming=True)
            
            # 시나리오 4: 컨텍스트 유지 대화
            print(f"\n🔗 시나리오 4: 이전 내용 참조")
            message4 = "앞에서 말한 특징 중 첫 번째 특징에 대해 더 자세히 설명해주세요."
            print(f"사용자: {message4}")
            response4 = await assistant.chat(message4)
            print(f"AI: {response4}")
            
            # 대화 히스토리 출력
            print(f"\n📜 대화 히스토리:")
            history = assistant.get_conversation_history()
            for i, entry in enumerate(history, 1):
                print(f"{i}. {entry}")
            
            # 대화 요약
            print(f"\n📋 대화 요약:")
            summary = await assistant.get_summary()
            print(summary)
            
            print("\n" + "=" * 60)
            print("✅ 과제 4 테스트 완료!")
            print("\n💡 추가 개선 아이디어:")
            print("  - 대화 저장/불러오기 기능")
            print("  - 다양한 페르소나 지원")
            print("  - 멀티모달 입력 지원")
            print("  - 실시간 웹 검색 통합")
            
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print("💡 ComprehensiveAssistant 클래스의 모든 메서드를 구현했는지 확인해주세요.")

# 테스트 실행
# await test_homework_4()  # 주석을 해제하고 실행하세요

## 15. 과제 제출 가이드

### ✅ 완료 체크리스트

과제를 완성했다면 아래 항목들을 확인해보세요:

#### 과제 1: 사용자 정의 함수 도구 만들기
- [ ] `get_exchange_rate()` 함수 구현 완료
- [ ] `calculate_tip()` 함수 구현 완료
- [ ] 두 함수가 AI와 정상적으로 연동됨
- [ ] 테스트 코드가 오류 없이 실행됨

#### 과제 2: 스트리밍 응답으로 실시간 번역기 만들기
- [ ] `translate_text()` 함수 구현 완료
- [ ] `StreamingTranslator` 클래스 구현 완료
- [ ] 스트리밍 방식으로 실시간 출력됨
- [ ] 여러 언어 쌍 번역 테스트 완료

#### 과제 3: 구조화된 응답으로 여행 계획 생성하기
- [ ] `TravelPlan` Pydantic 모델 정의 완료
- [ ] `get_destination_info()` 함수 구현 완료
- [ ] 구조화된 응답이 올바르게 파싱됨
- [ ] 여행 계획 필드들이 적절하게 출력됨

#### 과제 4: 대화형 AI 어시스턴트 만들기
- [ ] `ComprehensiveAssistant` 클래스 구현 완료
- [ ] 대화 히스토리 관리 기능 구현
- [ ] 스트리밍/비스트리밍 모드 지원
- [ ] 에러 처리 및 재시도 로직 구현
- [ ] 대화 요약 기능 구현
- [ ] 모든 시나리오 테스트 통과

### 📝 제출 방법

1. **코드 검증**: 모든 테스트 함수를 실행하여 오류가 없는지 확인
2. **주석 작성**: 구현한 코드에 간단한 설명 주석 추가
3. **실행 결과**: 각 과제의 테스트 출력 결과 확인
4. **선택 사항**: GitHub에 업로드하거나 팀과 공유

### 💡 문제 해결 팁

과제를 진행하면서 막히는 부분이 있다면:

1. **노트북 예제 참고**: 해당 기능을 사용한 예제를 다시 읽어보세요
2. **에러 메시지 확인**: 오류 메시지를 자세히 읽고 문제를 파악하세요
3. **단계별 접근**: 복잡한 과제는 작은 단위로 나누어 구현하세요
4. **print 디버깅**: 중간 결과를 출력하여 로직을 검증하세요
5. **환경 변수 확인**: API 키와 엔드포인트가 올바르게 설정되었는지 확인하세요

### 🎯 추가 도전 과제 (선택)

기본 과제를 완료했다면 다음 도전 과제에도 도전해보세요:

1. **과제 1+**: 실시간 환율 API 연동하기
2. **과제 2+**: 음성 입력/출력 기능 추가하기
3. **과제 3+**: 실제 여행 API(날씨, 호텔 등)와 연동하기
4. **과제 4+**: 웹 UI(Streamlit, Gradio)로 만들기
5. **통합 프로젝트**: 모든 기능을 합친 종합 애플리케이션 개발

### 🏆 성공적인 완료를 위한 조언

- **천천히 진행**: 서두르지 말고 한 단계씩 확실하게 이해하며 진행하세요
- **실험 정신**: 예제를 그대로 따라하기보다는 자신만의 방식으로 변형해보세요
- **에러는 친구**: 에러를 두려워하지 말고 학습의 기회로 활용하세요
- **문서 활용**: Azure AI Agent Framework 공식 문서를 참고하세요
- **커뮤니티**: 막히는 부분이 있다면 동료나 커뮤니티에 질문하세요

**즐거운 코딩 되세요! 🚀**